In [1]:
import os, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import torch
print(os.getpid())
sys.path.append(r"/lab/weissman_imaging/puzheng/Softwares")

3743443


In [4]:
library_master_folder = r'/lab/solexa_weissman/puzheng/MERFISH_Probes'
print(f"- FISH probe libraries stored in folder: {library_master_folder}")
# Load libraries
mf2_library_folder = os.path.join(library_master_folder, '4T1_MERFISH', 'MF2_4T1Primary')
mf2_probe_filename = os.path.join(mf2_library_folder, '2023_8_28_twist_merged_probes.xlsx')
mf2_probe_df = pd.read_excel(mf2_probe_filename)


- FISH probe libraries stored in folder: /lab/solexa_weissman/puzheng/MERFISH_Probes


In [6]:
# Load libraries
mf3_library_folder = os.path.join(library_master_folder, 'Brain_MS_immune', 'MF3-msBrain_immune_redesign')
mf3_probe_filename = os.path.join(mf3_library_folder, '2023_8_29_twist_merged_probes.xlsx')
mf3_probe_df = pd.read_excel(mf3_probe_filename)

In [7]:
merged_probe_df = pd.concat([mf2_probe_df, mf3_probe_df])

In [50]:
# generate twist-allowed version:
twist_probe_df = {'name':[], 'sequence':[]}
for _i, _row in merged_probe_df.iterrows():
    twist_probe_df['name'].append(_row['library'].replace('_','-') + '_' + _row['name'])
    twist_probe_df['sequence'].append(_row['sequence'])
twist_probe_df = pd.DataFrame(twist_probe_df)

In [51]:
twist_probe_df

,name,sequence
0,MF2-4T1-primary_Lck_ENSMUST00000167288.8_shift...,CGCAAACTGGTGCGGAAGGCGGATATCGACCGGAAGTCCAGGATAT...
1,MF2-4T1-primary_Mrpl37_ENSMUST00000030365.6_sh...,CGCAAACTGGTGCGGAAGGCGGATATCGACCGGAAGTCCAGGATAT...
2,MF2-4T1-primary_Mcat_ENSMUST00000061882.10_shi...,CGCAAACTGGTGCGGAAGGCGGATATCGACCGGAAGTCCAGGATAT...
3,MF2-4T1-primary_Il7r_ENSMUST00000228782.2_shif...,CGCAAACTGGTGCGGAAGGCGGATATCGACCGGAAGTCCAGGATAT...
4,MF2-4T1-primary_Ctu1_ENSMUST00000038332.9_shif...,CGCAAACTGGTGCGGAAGGCGGATATCGACCGGAAGTCCAGGATAT...
...,...,...
97421,MF3-msBrain-immune_Slc4a8_ENSMUST00000023776.1...,CCCGCAATGGCTGACAACCGCCCATGATCGTCCGATCTGGCCCATG...
97422,MF3-msBrain-immune_Mgam_ENSMUST00000238599.2_s...,CCCGCAATGGCTGACAACCGCCCATGATCGTCCGATCTGGCCCATG...
97423,MF3-msBrain-immune_Slc4a8_ENSMUST00000023776.1...,CCCGCAATGGCTGACAACCGCCCATGATCGTCCGATCTGGCCCATG...
97424,MF3-msBrain-immune_Zfp318_ENSMUST00000113481.9...,CCCGCAATGGCTGACAACCGCCCATGATCGTCCGATCTGGCCCATG...


## Plot summaries

In [46]:
libraries = np.unique(merged_probe_df['library'])
library_2_info = {}
for _lib in libraries:
    _lib_df = merged_probe_df.loc[merged_probe_df['library']==_lib]
    
    # generate info dataframe:
    _lib_info = {
        'gene':[],
        'probeNum': [],
        'readouts': [],
    }
    for _i, _pb in _lib_df.iterrows():
        _gene = _pb['name'].split('_')[0]
        _readout = _pb['name'].split(':')[-1]
        if _gene not in _lib_info['gene']:
            _lib_info['gene'].append(_gene)
            _lib_info['probeNum'].append(1)
            _lib_info['readouts'].append([_readout])
        else:
            _gene_index = _lib_info['gene'].index(_gene)
            _lib_info['probeNum'][_gene_index] += 1
            if _readout not in _lib_info['readouts'][_gene_index]:
                _lib_info['readouts'][_gene_index].append(_readout)
    # convert into df
    library_2_info[_lib] = pd.DataFrame(_lib_info)
    print(_lib, len(_lib_df), len(library_2_info[_lib]))

MF2-4T1_primary 63474 714
MF3-msBrain_immune 33952 368


In [52]:
save_folder = mf2_library_folder

merged_probe_filename = os.path.join(save_folder, 'mf2_mf3_probe.xlsx')
print(merged_probe_filename)
twist_probe_df.to_excel(merged_probe_filename, index=None)

merged_info_filename = os.path.join(save_folder, 'mf2_mf3_design_info.xlsx')
print(merged_info_filename)
# loop through libraries:
writer = pd.ExcelWriter(merged_info_filename, engine = 'xlsxwriter')
for _lib, _df in library_2_info.items():
    _df.to_excel(writer, index=None, sheet_name=_lib)
writer.close()

/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF2_4T1Primary/mf2_mf3_probe.xlsx
/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF2_4T1Primary/mf2_mf3_design_info.xlsx


In [37]:
_lib_info

,gene,probeNum,readouts
0,Lck,92,"[Stv_90, Stv_37, Stv_36, Stv_33]"
1,Mrpl37,91,"[Stv_90, Stv_60, Stv_50, Stv_29]"
2,Mcat,92,"[Stv_90, Stv_88, Stv_48, Stv_33]"
3,Il7r,86,"[Stv_90, Stv_87, Stv_45, Stv_31]"
4,Ctu1,90,"[Stv_90, Stv_37, Stv_32, Stv_31]"
...,...,...,...
709,Mmp9,94,"[Stv_37, Stv_33, Stv_30, Stv_29]"
710,Cxcr6,90,"[Stv_36, Stv_35, Stv_33, Stv_29]"
711,Pdcd1,87,"[Stv_36, Stv_35, Stv_32, Stv_30]"
712,Lamc2,91,"[Stv_36, Stv_33, Stv_32, Stv_31]"


In [31]:
_lib_info['gene'].index(_gene)

249

In [21]:
_lib_gene_2_pbNum = {_g:len(_pbs) for _g, _pbs in _lib_gene_2_pbs.items()}

In [22]:

_lib_gene_2_pbNum

{'Idh3g': 89,
 'Fcrla': 93,
 'Gimap1': 91,
 'Csmd3': 93,
 'Gna15': 96,
 'Zbtb18': 89,
 'Slc41a2': 91,
 'Tbx21': 93,
 'Il1a': 95,
 'Csf2': 96,
 'BE692007': 92,
 'Cd79a': 91,
 'Tigit': 91,
 'Slamf6': 92,
 'Adgre1': 91,
 'Tex2': 94,
 'Igf1r': 95,
 'Prdm1': 94,
 'Ms4a8a': 94,
 'Ly6g': 96,
 'Eepd1': 93,
 'Csf3r': 92,
 'Tax1bp3': 96,
 'Pglyrp1': 90,
 'Osgep': 95,
 'Mpp7': 93,
 'Irf4': 93,
 'Ncoa1': 93,
 'Relt': 91,
 '2310001H17Rik': 94,
 'Sh2d2a': 93,
 'Aicda': 93,
 'Flt3': 90,
 'Tcf7l2': 92,
 'Adgrg1': 88,
 'Tmem51': 94,
 'Rtcb': 94,
 'Ptprc': 92,
 'Agpat4': 94,
 'Pgap1': 90,
 'Siglecg': 90,
 'Cyth3': 90,
 'Gimap8': 93,
 'Rnf144a': 94,
 'Echs1': 91,
 'P2ry13': 93,
 'Ppp1r3b': 93,
 'Fscn1': 94,
 'Man1a': 95,
 'Dram1': 88,
 'Prf1': 90,
 'Ankrd33b': 92,
 'Stard9': 91,
 'Ebp': 93,
 'Rabgap1l': 95,
 'Pdha1': 92,
 'Adap2': 95,
 'Adam11': 93,
 'Map4k1': 94,
 'Zmiz2': 88,
 'Usp2': 92,
 'Tmem154': 91,
 'Slc4a8': 90,
 'Ssh1': 91,
 'Arhgef37': 94,
 'Plagl2': 89,
 'Pdzd4': 92,
 'Dip2b': 91,
 'Pacsin1':

In [26]:
_lib_df.iloc[0]['name'].split(':')

['Lck_ENSMUST00000167288.8_shift',
 '0_GC',
 '56.6_Tm',
 '58.0_specificity',
 '1_isospecificity',
 '0.33_Stv_90',
 'Stv_90',
 '',
 'Stv_90',
 'Stv_90']